In [1]:
# import pandas as pd
import pandas_ta
import numpy as np
 
# from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL
import matplotlib.pyplot as plt

In [2]:
import os
import time
import datetime
from six.moves import range

import tensorflow as tf

In [3]:
import features 

PRICE_COLUMN = 'close'
USE_PRICE_RANGE_COLUMNS = False

ranges_dict_path = 'data\\ranges_dict.pickle'
save_path = '.\\data\\featured_prices.csv'
prices_path = '.\\data\\prices_freq-min_2019-01-01_2019-03-28.csv'

data, ranges_dict = features.main(
    prices_path=prices_path,
    ranges_dicth_path=ranges_dict_path,
    save_path=save_path,
    onlyRead=True,
    cleanNans=True,
)

In [4]:
FEATURE_COLUMNS = []
for key in ranges_dict:
    FEATURE_COLUMNS += ranges_dict[key]['cols'] if ranges_dict[key]['normalize'] else []

In [5]:
if USE_PRICE_RANGE_COLUMNS:

    diff_cols = len(ranges_dict['prices']['cols']) - len(FEATURE_COLUMNS) - int(POSITION_AS_OBSERVATION)
    print(f'Difference of {diff_cols} columns between prices cols and normalized cols')
    print('In order to use Group Normalization Layer with 2 groups, both groups should be equal and sorted to be one first and then the other.')

    if diff_cols > 0:
        remove_cols = ['LR_14']
        print(f'The following columns are going to be removed: {remove_cols}')
        prices_cols = [col for col in ranges_dict['prices']['cols'] if col not in remove_cols]
    else:
        prices_cols = ranges_dict['prices']['cols']

    # Add prices cols into the FEATURE_COLUMNS
    FEATURE_COLUMNS = prices_cols + FEATURE_COLUMNS

# Make sure that PRICE_COL is in data
ALL_COLS = [PRICE_COLUMN] if PRICE_COLUMN not in FEATURE_COLUMNS else []
ALL_COLS += FEATURE_COLUMNS

# Set the columns used in data PRICE_COL + FEATURE_COLS
data = data[ALL_COLS]

In [6]:
assert not np.isinf(data).any(1).any(), data[np.isinf(data).any(1)]
assert not data.isnull().any().any()

In [7]:
unit_factor = 60*24*30 # months 
print(f'Data for {len(data.index) / unit_factor:.3f} units')

Data for 2.842 units


In [8]:
train_time = 2
gap_time = 0.3
test_time = len(data.index) / unit_factor - train_time - gap_time

train_end = int(train_time * unit_factor)
test_start = train_end + int(gap_time * unit_factor)
test_end = test_start + int(test_time * unit_factor)

train = data.iloc[0:train_end, :]
test = data.iloc[test_start:test_end, :]

In [9]:
from own_stock_env import OwnStocksEnv

steps_per_episode = 60
window_size = 10
POSITION_AS_OBSERVATION = True

SEED = 12345

#### ONLY FOR TESTING OVERFITING

# train = train[0:steps_per_episode*2]
# test = test[0:steps_per_episode*2]

# train = train[0:steps_per_episode*20]
# test = test[0:steps_per_episode*20]

##############################################

train_env = OwnStocksEnv(
    df=train,
    window_size=window_size,
    frame_bound=(window_size, len(train)),
    steps_per_episode=steps_per_episode,
    is_training=True,
    constant_step=False,
    position_as_observation=POSITION_AS_OBSERVATION,
)
train_env.seed(SEED);

test_env = OwnStocksEnv(
    df=test,
    window_size=window_size,
    frame_bound=(window_size, len(test)),
    steps_per_episode=steps_per_episode,
    is_training=True,
    constant_step=False,
    position_as_observation=POSITION_AS_OBSERVATION,
)
test_env.seed(SEED);

In [10]:
# Transform Gym Environment to TFPyEnvironment
from gym import spaces
from tf_agents.environments import tf_py_environment
from tf_agents.environments.gym_wrapper import GymWrapper

# Otherwise raise error on evaluating ChosenActionHistogram metric
spec_dtype_map = {spaces.Discrete: np.int32}

tf_env = tf_py_environment.TFPyEnvironment(GymWrapper(train_env, spec_dtype_map=spec_dtype_map))
eval_tf_env = tf_py_environment.TFPyEnvironment(GymWrapper(test_env, spec_dtype_map=spec_dtype_map))

In [11]:
from absl import logging

logging.set_verbosity(logging.INFO)
tf.compat.v1.enable_v2_behavior()

In [12]:
from tensorflow.keras.optimizers import Adam, SGD
from tf_agents.utils import common

# Params for train
num_iterations = 3000000
num_iterations = min(train_env.frame_bound[-1], num_iterations)

initial_collect_steps = num_iterations // 1000 # 1000
replay_buffer_capacity = num_iterations
collect_steps_per_iteration = 1
# TODO: Use other kind of policy like Boltzam?
epsilon_greedy = 0.1

target_update_tau = 0.05
target_update_period = 5

# TODO: Improve learning rate with schedule and on e-greedy too
train_steps_per_iteration = 1
n_step_update = 1
batch_size = 32
learning_rate = 3e-4
optimizer = SGD(learning_rate=learning_rate) # Adam(learning_rate=learning_rate)
gradient_clipping = None

td_errors_loss_fn = common.element_wise_huber_loss # common.element_wise_squared_loss # common.element_wise_huber_loss

gamma = 0.99
reward_scale_factor = 1.0

use_tf_functions = True

# Params for eval
num_eval_episodes = test_env.frame_bound[-1] // test_env.steps_per_episode
eval_interval = num_iterations // 25 # 1000
eval_interval = max(eval_interval, steps_per_episode)
eval_metrics_callback = None

# Params for checkpoints
train_checkpoint_interval = num_iterations // 10 # 10000
policy_checkpoint_interval = num_iterations // 20 # 5000
rb_checkpoint_interval = num_iterations // 5 # 20000

# Params for summaries and logging
log_interval = num_iterations // 100 # 1000
log_interval = max(log_interval, steps_per_episode)
summaries_flush_secs = 10
summary_interval = num_iterations // 100 # 1000
summary_interval = max(summary_interval, steps_per_episode)
debug_summaries = False
summarize_grads_and_vars = True

In [13]:
# !rmdir /s /q .\\logs\\dqn

In [14]:
TRAIN_MODEL = True

root_dir = 'logs\\dqn'

if TRAIN_MODEL:
    root_dir = os.path.join(root_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
else:
    root_dir = os.path.join(root_dir, '20200503-095638')

In [15]:
from tf_agents.metrics import tf_metrics

root_dir = os.path.expanduser(root_dir)
train_dir = os.path.join(root_dir, 'train')
eval_dir = os.path.join(root_dir, 'eval')

train_summary_writer = tf.summary.create_file_writer(
    train_dir, flush_millis=summaries_flush_secs * 1000)
train_summary_writer.set_as_default()
train_metrics = [
    # tf_metrics.NumberOfEpisodes(),
    # tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    # tf_metrics.AverageEpisodeLengthMetric(),
    # tf_metrics.ChosenActionHistogram(dtype=tf.int32),
]

eval_summary_writer = tf.summary.create_file_writer(
    eval_dir, flush_millis=summaries_flush_secs * 1000)
eval_metrics = [
    tf_metrics.AverageReturnMetric(buffer_size=num_eval_episodes),
    # tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes)
]

global_step = tf.compat.v1.train.get_or_create_global_step()

In [16]:
# Define Q-network

from tf_agents.networks import q_network
from tf_agents.networks import q_rnn_network

train_sequence_length = window_size

if train_sequence_length > 1:
    input_fc_layer_params = (8,)
    lstm_size = (16,)
    output_fc_layer_params = (8,)
else:
    fc_layer_params = (100,)

if train_sequence_length != 1 and n_step_update != 1:
    raise NotImplementedError(
        'Currently not supporting n-step updates with stateful networks (i.e., RNNs)')

if train_sequence_length > 1:
    q_net = q_rnn_network.QRnnNetwork(
        tf_env.observation_spec(),
        tf_env.action_spec(),
        input_fc_layer_params=input_fc_layer_params,
        lstm_size=lstm_size,
        output_fc_layer_params=output_fc_layer_params
    )
else:
    q_net = q_network.QNetwork(
        tf_env.observation_spec(),
        tf_env.action_spec(),
        fc_layer_params=fc_layer_params
    )
    train_sequence_length = n_step_update

In [35]:
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common

with tf.summary.record_if(
    lambda: tf.math.equal(global_step % summary_interval, 0)):

    # TODO(b/127301657): Decay epsilon based on global step, cf. cl/188907839
    tf_agent = dqn_agent.DqnAgent(
        tf_env.time_step_spec(),
        tf_env.action_spec(),
        q_network=q_net,
        epsilon_greedy=epsilon_greedy,
        n_step_update=n_step_update,
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        optimizer=optimizer,
        td_errors_loss_fn=td_errors_loss_fn,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        gradient_clipping=gradient_clipping,
        debug_summaries=debug_summaries,
        summarize_grads_and_vars=summarize_grads_and_vars,
        train_step_counter=global_step)
    tf_agent.initialize()

    eval_policy = tf_agent.policy
    collect_policy = tf_agent.collect_policy

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec=tf_agent.collect_data_spec,
        batch_size=tf_env.batch_size,
        max_length=replay_buffer_capacity)

    if TRAIN_MODEL:
      collect_driver = dynamic_step_driver.DynamicStepDriver(
          tf_env,
          collect_policy,
          observers=[replay_buffer.add_batch] + train_metrics,
          num_steps=collect_steps_per_iteration)

    train_checkpointer = common.Checkpointer(
        ckpt_dir=train_dir,
        agent=tf_agent,
        global_step=global_step,
        metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))
    policy_checkpointer = common.Checkpointer(
        ckpt_dir=os.path.join(train_dir, 'policy'),
        policy=eval_policy,
        global_step=global_step)
    rb_checkpointer = common.Checkpointer(
        ckpt_dir=os.path.join(train_dir, 'replay_buffer'),
        max_to_keep=1,
        replay_buffer=replay_buffer)

    train_checkpointer.initialize_or_restore()
    rb_checkpointer.initialize_or_restore()

    if TRAIN_MODEL:

      if use_tf_functions:
        # To speed up collect use common.function.
        collect_driver.run = common.function(collect_driver.run)
        tf_agent.train = common.function(tf_agent.train)

      initial_collect_policy = random_tf_policy.RandomTFPolicy(
          tf_env.time_step_spec(), tf_env.action_spec())

      # Collect initial replay data.
      logging.info(
          'Initializing replay buffer by collecting experience for %d steps with '
          'a random policy.', initial_collect_steps)
      dynamic_step_driver.DynamicStepDriver(
          tf_env,
          initial_collect_policy,
          observers=[replay_buffer.add_batch] + train_metrics,
          num_steps=initial_collect_steps).run()

      # Initial eval metric
      logging.info(
          f'Initial eval metric on {num_eval_episodes} episodes'
      )
      results = metric_utils.eager_compute(
          eval_metrics,
          eval_tf_env,
          eval_policy,
          num_episodes=num_eval_episodes,
          train_step=global_step,
          summary_writer=eval_summary_writer,
          summary_prefix='Metrics',
      )
      if eval_metrics_callback is not None:
        eval_metrics_callback(results, global_step.numpy())
      metric_utils.log_metrics(eval_metrics)

      time_step = None
      policy_state = collect_policy.get_initial_state(tf_env.batch_size)

      timed_at_step = global_step.numpy()
      time_acc = 0

      # Dataset generates trajectories with shape [Bx2x...]
      logging.info(
          f'Dataset generates trajectories'
      )
      dataset = replay_buffer.as_dataset(
          num_parallel_calls=3,
          sample_batch_size=batch_size,
          # single_deterministic_pass=True,
          num_steps=train_sequence_length + 1).prefetch(3)
      iterator = iter(dataset)

      def train_step():
        experience, _ = next(iterator)
        return tf_agent.train(experience)

      if use_tf_functions:
        train_step = common.function(train_step)

      for _ in range(num_iterations):
        start_time = time.time()
        time_step, policy_state = collect_driver.run(
            time_step=time_step,
            policy_state=policy_state,
        )
        for _ in range(train_steps_per_iteration):
          train_loss = train_step()
        time_acc += time.time() - start_time

        if global_step.numpy() % log_interval == 0:
          logging.info('step = %d, loss = %f', global_step.numpy(),
                      train_loss.loss)
          steps_per_sec = (global_step.numpy() - timed_at_step) / time_acc
          logging.info('%.3f steps/sec', steps_per_sec)
          tf.compat.v2.summary.scalar(
              name='global_steps_per_sec', data=steps_per_sec, step=global_step)
          timed_at_step = global_step.numpy()
          time_acc = 0

        for train_metric in train_metrics:
          train_metric.tf_summaries(
              train_step=global_step, step_metrics=train_metrics[:2])

        if global_step.numpy() % train_checkpoint_interval == 0:
          train_checkpointer.save(global_step=global_step.numpy())

        if global_step.numpy() % policy_checkpoint_interval == 0:
          policy_checkpointer.save(global_step=global_step.numpy())

        if global_step.numpy() % rb_checkpoint_interval == 0:
          rb_checkpointer.save(global_step=global_step.numpy())

        if global_step.numpy() % eval_interval == 0:
          start_time = time.time()
          results = metric_utils.eager_compute(
              eval_metrics,
              eval_tf_env,
              eval_policy,
              num_episodes=num_eval_episodes,
              train_step=global_step,
              summary_writer=eval_summary_writer,
              summary_prefix='Metrics',
          )
          logging.info(
            f'Eval time equal to {time.time() - start_time:.0f}'
          )
          if eval_metrics_callback is not None:
            eval_metrics_callback(results, global_step.numpy())
          metric_utils.log_metrics(eval_metrics)

INFO:absl:Checkpoint available: logs\dqn\20200503-105726\train\ckpt-172800
INFO:absl:Checkpoint available: logs\dqn\20200503-105726\train\policy\ckpt-172800
INFO:absl:Checkpoint available: logs\dqn\20200503-105726\train\replay_buffer\ckpt-172800
INFO:absl:Initializing replay buffer by collecting experience for 86 steps with a random policy.
INFO:absl:Initial eval metric on 390 episodes
INFO:absl: 
		 AverageReturn = 0.27574947476387024
INFO:absl:Dataset generates trajectories
INFO:absl:step = 173664, loss = 0.002589
INFO:absl:18.123 steps/sec
INFO:absl:step = 174528, loss = 0.004590
INFO:absl:22.670 steps/sec
INFO:absl:step = 175392, loss = 0.002990
INFO:absl:20.331 steps/sec
INFO:absl:step = 176256, loss = 0.003260
INFO:absl:20.508 steps/sec
INFO:absl:Eval time equal to 27
INFO:absl: 
		 AverageReturn = 0.1085846945643425
INFO:absl:step = 177120, loss = 0.002411
INFO:absl:19.916 steps/sec
INFO:absl:Saved checkpoint: logs\dqn\20200503-105726\train\policy\ckpt-177120
INFO:absl:step = 17

KeyboardInterrupt: 

In [62]:
# num_episodes = 10

# val_env = OwnStocksEnv(
#     df=train,
#     window_size=window_size,
#     frame_bound=(window_size, len(train)),
#     steps_per_episode=steps_per_episode,
#     is_training=True,
#     constant_step=True,
#     position_as_observation=POSITION_AS_OBSERVATION,
# )
# val_tf_env = tf_py_environment.TFPyEnvironment(GymWrapper(val_env))
# val_env.seed(SEED)

# val_dir = os.path.join(root_dir, 'val')
# val_summary_writer = tf.summary.create_file_writer(
#     val_dir, flush_millis=summaries_flush_secs * 1000)
# val_metrics = [
#     tf_metrics.AverageReturnMetric(buffer_size=num_episodes),
#     # tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes)
#     tf_metrics.ChosenActionHistogram(buffer_size=num_episodes, dtype=tf.int32),
# ]

# val_metrics_callback = None

# print('Start evaluating')
# results = metric_utils.eager_compute(
#     val_metrics,
#     val_tf_env,
#     tf_agent.policy,
#     num_episodes=num_episodes,
#     train_step=global_step,
#     # summary_writer=val_summary_writer,
#     # summary_prefix='Metrics',
# )
# if val_metrics_callback is not None:
#     val_metrics_callback(results, global_step.numpy())
# metric_utils.log_metrics(val_metrics)

Start evaluating
INFO:absl: 
		 AverageReturn = 0.06606774032115936
		 ChosenActionHistogram = [0 0 0 0 0 1 0 0 0 0]


In [18]:
env_data = test

all_envs = {}

full_env = OwnStocksEnv(
    df=env_data,
    window_size=window_size,
    frame_bound=(window_size, len(env_data)),
    steps_per_episode=len(env_data) - window_size, # steps_per_episode,
    constant_step=True,
    is_training=False,
    position_as_observation=POSITION_AS_OBSERVATION,
)
all_envs['Full test'] = full_env

#TODO: For the is_training=True we have to make that all executions are using same cases
step_env = OwnStocksEnv(
    df=env_data,
    window_size=window_size,
    frame_bound=(window_size, len(env_data)),
    steps_per_episode=steps_per_episode,
    constant_step=True,
    is_training=True,
    position_as_observation=POSITION_AS_OBSERVATION,
)
all_envs[f'Test step of {steps_per_episode}'] = step_env

large_step_env = OwnStocksEnv(
    df=env_data,
    window_size=window_size,
    frame_bound=(window_size, len(env_data)),
    steps_per_episode=10 * steps_per_episode,
    constant_step=True,
    is_training=True,
    position_as_observation=POSITION_AS_OBSERVATION,
)
all_envs[f'Test step of {10*steps_per_episode}'] = large_step_env

small_step_env = OwnStocksEnv(
    df=env_data,
    window_size=window_size,
    frame_bound=(window_size, len(env_data)),
    steps_per_episode=int(0.1 * steps_per_episode),
    constant_step=True,
    is_training=True,
    position_as_observation=POSITION_AS_OBSERVATION,
)
all_envs[f'Test step of {int(0.1 * steps_per_episode)}'] = small_step_env

In [63]:
from own_stock_env import runAllTestEnv

In [20]:
# Apply random policy on env
runAllTestEnv(all_envs, select_action_func=full_env.action_space.sample);

Testing enviorment Full test:
Total rewards: -32.94 ± 0.000 (mean ± std. dev. of 1 iterations)
Total profits: -0.89% ± 0.000% (mean ± std. dev. of 1 iterations)
--------------------------------------------------
Testing enviorment Test step of 60:
Total rewards: 0.40 ± 7.905 (mean ± std. dev. of 390 iterations)
Total profits: 0.01% ± 0.203% (mean ± std. dev. of 390 iterations)
--------------------------------------------------
Testing enviorment Test step of 600:
Total rewards: 13.13 ± 31.224 (mean ± std. dev. of 39 iterations)
Total profits: 0.33% ± 0.794% (mean ± std. dev. of 39 iterations)
--------------------------------------------------
Testing enviorment Test step of 6:
Total rewards: -0.08 ± 2.585 (mean ± std. dev. of 3900 iterations)
Total profits: -0.00% ± 0.071% (mean ± std. dev. of 3900 iterations)
--------------------------------------------------


In [21]:
# Applying long term policy (buy at initial and do not sell) on env
from gym_anytrading.envs import Actions 

def always_buy_func():
    return  Actions.Buy.value

runAllTestEnv(all_envs, select_action_func=always_buy_func);

Testing enviorment Full test:
Total rewards: 0.00 ± 0.000 (mean ± std. dev. of 1 iterations)
Total profits: 4.82% ± 0.000% (mean ± std. dev. of 1 iterations)
--------------------------------------------------
Testing enviorment Test step of 60:
Total rewards: 0.00 ± 0.000 (mean ± std. dev. of 390 iterations)
Total profits: 0.02% ± 0.213% (mean ± std. dev. of 390 iterations)
--------------------------------------------------
Testing enviorment Test step of 600:
Total rewards: 0.00 ± 0.000 (mean ± std. dev. of 39 iterations)
Total profits: 0.38% ± 0.850% (mean ± std. dev. of 39 iterations)
--------------------------------------------------
Testing enviorment Test step of 6:
Total rewards: 0.00 ± 0.000 (mean ± std. dev. of 3900 iterations)
Total profits: -0.00% ± 0.088% (mean ± std. dev. of 3900 iterations)
--------------------------------------------------


In [75]:
# Applying baseline policy on env
# Manual policy used as baseline
from gym_anytrading.envs import Positions, Actions

rsi_col = 'RSI_14'
# rsi_col = 'Close_rsi'
rsi_index = full_env.feature_columns.index(rsi_col)

# RSI usually is between 0 and 100, here is normalized between -1 and 1
# The baseline strategy is buy at 30 and sell at 70 otherwise hold
def select_baseline_action(observation, rsi_thresh_buy=-0.6, rsi_thresh_sell=0.4, rsi_index=rsi_index):
    # Use only last observation
    obs = observation[-1]

    position_value = int(obs[-1])
    rsi = obs[rsi_index]

    if position_value == Positions.Short.value and rsi <= rsi_thresh_buy:
        action = Actions.Buy.value
    elif position_value == Positions.Long.value and rsi >= rsi_thresh_sell:
        action = Actions.Sell.value
    else:
        # Hold
        # if it was in short remain in short because is selling
        # if it was in long remain in long because is buying
        action = position_value
    
    return action

runAllTestEnv(all_envs, select_action_func=select_baseline_action, use_observation=True, rsi_thresh_buy=0.4, rsi_thresh_sell=0.6);

Testing enviorment Full test:
Total rewards: 179.64 ± 0.000 (mean ± std. dev. of 1 iterations)
Total profits: 4.70% ± 0.000% (mean ± std. dev. of 1 iterations)
--------------------------------------------------
Testing enviorment Test step of 60:
Total rewards: 1.62 ± 3.036 (mean ± std. dev. of 390 iterations)
Total profits: 0.01% ± 0.142% (mean ± std. dev. of 390 iterations)
--------------------------------------------------
Testing enviorment Test step of 600:
Total rewards: 8.23 ± 13.574 (mean ± std. dev. of 39 iterations)
Total profits: 0.19% ± 0.343% (mean ± std. dev. of 39 iterations)
--------------------------------------------------
Testing enviorment Test step of 6:
Total rewards: 0.01 ± 0.158 (mean ± std. dev. of 3900 iterations)
Total profits: 0.00% ± 0.049% (mean ± std. dev. of 3900 iterations)
--------------------------------------------------


In [31]:
def select_TFEnv_action(TFEnv, policy, done, time_step=None, policy_state=None):
    
    action_step = policy.action(time_step, policy_state)

    # TODO(b/134487572): TF2 while_loop seems to either ignore
    # parallel_iterations or doesn't properly propagate control dependencies
    # from one step to the next. Without this dep, self.env.step() is called
    # in parallel.
    with tf.control_dependencies(tf.nest.flatten([time_step])):
        next_time_step = TFEnv.step(action_step.action)

    policy_state = action_step.state

    action = action_step.action.numpy()[0]

    done = next_time_step.discount.numpy()[0] == 0

    return action, done, next_time_step, policy_state

In [32]:
all_tf_envs = {}

for key, value in all_envs.items():
    all_tf_envs[key] = tf_py_environment.TFPyEnvironment(GymWrapper(value))

In [33]:
runAllTestEnv(all_tf_envs, select_action_func=select_TFEnv_action, use_model=True, isTFEnv=True, policy=tf_agent.policy)

Testing enviorment Full test:
Total rewards: 67.97 ± 0.000 (mean ± std. dev. of 1 iterations)
Total profits: 1.76% ± 0.000% (mean ± std. dev. of 1 iterations)
--------------------------------------------------
Testing enviorment Test step of 60:
Total rewards: 0.27 ± 3.780 (mean ± std. dev. of 390 iterations)
Total profits: 0.01% ± 0.097% (mean ± std. dev. of 390 iterations)
--------------------------------------------------
Testing enviorment Test step of 600:
Total rewards: 0.92 ± 12.293 (mean ± std. dev. of 39 iterations)
Total profits: 0.02% ± 0.316% (mean ± std. dev. of 39 iterations)
--------------------------------------------------
Testing enviorment Test step of 6:
Total rewards: -0.01 ± 2.990 (mean ± std. dev. of 3900 iterations)
Total profits: -0.00% ± 0.077% (mean ± std. dev. of 3900 iterations)
--------------------------------------------------
